Мы будем работать со следующими столбцами:

        date — дата наблюдения;
        province/state — наименование провинции/штата;
        country — наименование страны;
        confirmed — общее число зафиксированных случаев на указанный день;
        deaths — общее число зафиксированных смертей на указанный день;
        recovered — общее число выздоровлений на указанный день.


In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
import pandas as pd
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.tail(3))

,date,province/state,country,confirmed,deaths,recovered
306426,05/29/2021,Zhejiang,China,1364.0,1.0,1324.0
306427,05/29/2021,Zhytomyr Oblast,Ukraine,87550.0,1738.0,83790.0
306428,05/29/2021,Zuid-Holland,Netherlands,391559.0,4252.0,0.0


Данная таблица содержит следующие столбцы:

        country — наименование страны;
        date — дата наблюдения;
        total_vaccinations — общее число введённых вакцин в стране на указанный день;
        people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
        people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как 

);
people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как
);
daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
vaccines — комбинации вакцин, используемые в стране.

In [25]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data [
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']]

display(vaccinations_data.tail(3))

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
42792,Zimbabwe,2021-09-03,4372216.0,2681657.0,17.77,1690559.0,11.20,42317.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
42793,Zimbabwe,2021-09-04,4400246.0,2698332.0,17.88,1701914.0,11.28,41413.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
42794,Zimbabwe,2021-09-05,4431599.0,2713248.0,17.98,1718351.0,11.39,41964.0,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."


Предобработка данных

Опираясь на замечания выше, выполним небольшую предобработку.

В таблице covid_data:	

Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:
Преобразуем даты в формат datetime с помощью функции pd.to_datetime():
Создадим признак больных на данный момент (active). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:
Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():

In [34]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()
covid_data['date'] = pd.to_datetime(covid_data['date'])
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

display(covid_data['date'].min())
display(covid_data['date'].max())

Timestamp('2020-01-22 00:00:00')

Timestamp('2021-05-29 00:00:00')

В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:

In [68]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])
display(vaccinations_data['date'].min())
display(vaccinations_data['date'].max())

Timestamp('2020-12-02 00:00:00')

Timestamp('2021-09-06 00:00:00')

С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.

Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, вычисленный в задании 3.1. То есть в результирующую таблицу должны попасть все записи из таблицы covid_data и из её пересечения с vaccinations_data, но не более. Результат объединения занесите в переменную covid_df.

In [82]:
 
covid_df = covid_data.merge(vaccinations_data, how = 'left', on = ['date', 'country'])
    
#covid_df.sort_values(by='date')
display(covid_df.info())

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
39389,2020-01-22,Japan,2.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33822,2020-01-22,Hong Kong,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16251,2020-01-22,China,547.0,17.0,28.0,502.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77965,2020-01-22,Thailand,4.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76636,2020-01-22,Taiwan,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69100,2021-05-29,Serbia,712046.0,6844.0,0.0,705202.0,274.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,36912.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
61856,2021-05-29,Peru,1947555.0,68978.0,1897522.0,-18945.0,5501.0,162.0,4728.0,4148590.0,2885452.0,8.65,1263138.0,3.79,114612.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm..."
36630,2021-05-29,Iran,2893218.0,79741.0,2425033.0,388444.0,7107.0,173.0,14781.0,NaN,NaN,NaN,NaN,NaN,100393.0,"COVIran Barekat, Covaxin, Oxford/AstraZeneca, ..."
76635,2021-05-29,Syria,24440.0,1763.0,21593.0,1084.0,30.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,1660.0,"Johnson&Johnson, Oxford/AstraZeneca, Sinopharm..."


В получившейся в задании 3.3 таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность) и recover_rate — общий процент случаев выздоровления. Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.

In [85]:
covid_df['death_rate'] = (covid_df['deaths']/covid_df['confirmed'])*100
covid_df['recover_rate'] = (covid_df['recovered']/covid_df['confirmed'])*100
#covid_df.sort_values(by='date')

Какова максимальная летальность в США (United States) за весь период? Ответ округлите до второго знака после запятой.

In [93]:
round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2)

7.19

Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой.

In [97]:
round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)

67.06